## MLflow's Model Registry

In [92]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

### 2.5
update api: list_experiment to search_experiments

In [93]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/mlruns/4', creation_time=1717900888078, experiment_id='4', last_update_time=1717900888078, lifecycle_stage='active', name='demo-study-group-w2-part1', tags={}>,
 <Experiment: artifact_location='/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1717352208982, experiment_id='2', last_update_time=1717352208982, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/mlruns/0', creation_time=1717352186315, experiment_id='0', last_update_time=1717352186315, lifecycle_stage='active', name='Default', tags={}>]

In [94]:
#client.create_experiment(name="demo-study-group-w2-part1")

Let's check the latest versions for the experiment with id `1`...

In [95]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='4',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [96]:
runs

[<Run: data=<RunData: metrics={'best_iteration': 217.0,
  'rmse': 6.310178838640537,
  'stopped_iteration': 267.0,
  'validation-rmse': 6.307605634208257}, params={'custom_metric': 'None',
  'early_stopping_rounds': '50',
  'learning_rate': '0.30070237573310177',
  'max_depth': '62',
  'maximize': 'None',
  'min_child_weight': '1.7064946336172562',
  'num_boost_round': '1000',
  'objective': 'reg:linear',
  'reg_alpha': '0.012171939658517199',
  'reg_lambda': '0.3301990638226685',
  'seed': '42',
  'verbose_eval': 'True'}, tags={'mlflow.log-model.history': '[{"run_id": "26c7438ec7e042aea691d8701af225cf", '
                              '"artifact_path": "model", "utc_time_created": '
                              '"2024-06-09 03:33:32.359594", "flavors": '
                              '{"python_function": {"loader_module": '
                              '"mlflow.xgboost", "python_version": "3.10.14", '
                              '"data": "model.xgb", "env": {"conda": '
           

In [97]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 26c7438ec7e042aea691d8701af225cf, rmse: 6.3102
run id: a19b2e7924c2490f8c684886bc201463, rmse: 6.3302
run id: c62153746db94f589fdf45f1b83d6a80, rmse: 6.3302
run id: 310226654c6f4a69a29f8f35a55de79f, rmse: 6.3308
run id: a15c9471441f41878ef8c7fc0dcdb959, rmse: 6.3419


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [98]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [99]:
run_id = "26c7438ec7e042aea691d8701af225cf"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="demo-w2-p1")

Registered model 'demo-w2-p1' already exists. Creating a new version of this model...
Created version '4' of model 'demo-w2-p1'.


<ModelVersion: aliases=[], creation_timestamp=1717912671472, current_stage='None', description=None, last_updated_timestamp=1717912671472, name='demo-w2-p1', run_id='26c7438ec7e042aea691d8701af225cf', run_link=None, source='/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/mlruns/4/26c7438ec7e042aea691d8701af225cf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [100]:
model_name = "demo-w2-p1"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: Production
version: 4, stage: None


/var/folders/09/p_32v97x0m92g9kb_3v9t2wm0000gn/T/ipykernel_5937/2992258537.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [101]:
print(latest_versions)

[<ModelVersion: aliases=[], creation_timestamp=1717909487356, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-06-09', last_updated_timestamp=1717911862560, name='demo-w2-p1', run_id='26c7438ec7e042aea691d8701af225cf', run_link='', source='/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/mlruns/4/26c7438ec7e042aea691d8701af225cf/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>, <ModelVersion: aliases=[], creation_timestamp=1717911802236, current_stage='Production', description='The model version 3 was transitioned to Staging on 2024-06-09', last_updated_timestamp=1717912580871, name='demo-w2-p1', run_id='26c7438ec7e042aea691d8701af225cf', run_link=None, source='/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/mlruns/4/26c7438ec7e042aea691d8701af225cf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>, <ModelVe

In [102]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/var/folders/09/p_32v97x0m92g9kb_3v9t2wm0000gn/T/ipykernel_5937/1957332551.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717911802236, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-06-09', last_updated_timestamp=1717912671503, name='demo-w2-p1', run_id='26c7438ec7e042aea691d8701af225cf', run_link=None, source='/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/mlruns/4/26c7438ec7e042aea691d8701af225cf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### 2.5
After this, check the description of the model, should have:

The model version 3 was ... to Staging ...

In [103]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1717911802236, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-06-09', last_updated_timestamp=1717912671513, name='demo-w2-p1', run_id='26c7438ec7e042aea691d8701af225cf', run_link=None, source='/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/mlruns/4/26c7438ec7e042aea691d8701af225cf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [104]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(df):
    #df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [105]:
#df = read_dataframe("data/green_tripdata_2021-03.parquet")
df = pd.read_parquet('./data/green_tripdata_2021-03.parquet')
df = read_dataframe(df)

/var/folders/09/p_32v97x0m92g9kb_3v9t2wm0000gn/T/ipykernel_5937/3233001999.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [106]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/preprocessor'

In [107]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [108]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-03-01 00:05:42,2021-03-01 00:14:03,N,1.0,83,129,1.0,1.56,7.50,...,0.5,0.00,0.0,None,0.3,8.80,1.0,1.0,0.0,8.350000
1,2,2021-03-01 00:21:03,2021-03-01 00:26:17,N,1.0,243,235,1.0,0.96,6.00,...,0.5,0.00,0.0,None,0.3,7.30,2.0,1.0,0.0,5.233333
2,2,2021-03-01 00:02:06,2021-03-01 00:22:26,N,1.0,75,242,1.0,9.93,28.00,...,0.5,2.00,0.0,None,0.3,31.30,1.0,1.0,0.0,20.333333
3,2,2021-03-01 00:24:03,2021-03-01 00:31:43,N,1.0,242,208,1.0,2.57,9.50,...,0.5,0.00,0.0,None,0.3,10.80,2.0,1.0,0.0,7.666667
4,1,2021-03-01 00:11:10,2021-03-01 00:14:46,N,1.0,41,151,1.0,0.80,5.00,...,0.5,1.85,0.0,None,0.3,8.15,1.0,1.0,0.0,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83822,2,2021-03-31 22:07:00,2021-03-31 22:13:00,None,NaN,41,75,NaN,1.48,8.46,...,0.0,1.44,0.0,None,0.3,10.20,NaN,NaN,NaN,6.000000
83823,2,2021-03-31 22:56:00,2021-03-31 23:13:00,None,NaN,95,95,NaN,0.09,54.25,...,0.0,0.00,0.0,None,0.3,57.30,NaN,NaN,NaN,17.000000
83824,2,2021-03-31 22:36:00,2021-03-31 22:45:00,None,NaN,95,95,NaN,0.66,8.11,...,0.0,0.00,0.0,None,0.3,8.41,NaN,NaN,NaN,9.000000
83825,2,2021-03-31 23:35:00,2021-04-01 00:00:00,None,NaN,37,14,NaN,9.58,36.83,...,0.0,0.00,0.0,None,0.3,39.88,NaN,NaN,NaN,25.000000


In [113]:
X_test = preprocess(df, dv)

In [114]:
target = "duration"
y_test = df[target].values

In [115]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

/Users/gabriel/miniconda3/envs/mlops-w2/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/Users/gabriel/miniconda3/envs/mlops-w2/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:59:20] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 8.26 s, sys: 145 ms, total: 8.41 s
Wall time: 1.2 s


/Users/gabriel/miniconda3/envs/mlops-w2/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.26089964278947}

In [116]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

/var/folders/09/p_32v97x0m92g9kb_3v9t2wm0000gn/T/ipykernel_5937/684772386.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717911802236, current_stage='Production', description='The model version 3 was transitioned to Staging on 2024-06-09', last_updated_timestamp=1717912763968, name='demo-w2-p1', run_id='26c7438ec7e042aea691d8701af225cf', run_link=None, source='/Users/gabriel/workspace/selfstudy/mlops_zoomcamp/02-experiment-tracking/mlruns/4/26c7438ec7e042aea691d8701af225cf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [117]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/Users/gabriel/miniconda3/envs/mlops-w2/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/Users/gabriel/miniconda3/envs/mlops-w2/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:59:28] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 8.09 s, sys: 91.5 ms, total: 8.18 s
Wall time: 1.13 s


/Users/gabriel/miniconda3/envs/mlops-w2/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.26089964278947}